# Importing Libraries

In [1]:
!pip install efficientnet -q
!pip install pydrive -q

In [2]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import efficientnet.tfkeras as efn

from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import compute_class_weight

# Configuration

In [3]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy

In [4]:
COMPETITION_NAME = "siim-covid19-resized-to-512px-png"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16
GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)

Running on TPU: grpc://10.0.0.2:8470
Running on 8 replicas


In [5]:
class Config:
    IMG_SIZE = 512
    NUM_REPLICAS_IN_SYNC = strategy.num_replicas_in_sync
    BATCH_SIZE = 16 * NUM_REPLICAS_IN_SYNC
    GDC_DS_PATH = GCS_DS_PATH
    NUM_CLASSES = 4
    LR = 0.001 * NUM_REPLICAS_IN_SYNC
    EPOCHS = 20

In [6]:
config = Config()

# Loading data

In [7]:
study_df = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
study_df['study_id'] = study_df['id'].map(lambda x: x.split('_')[0])
study_df.drop(columns = 'id', inplace=True)
study_df.head(2)

,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,study_id
0,0,1,0,0,00086460a852
1,0,0,0,1,000c9c05fd14


In [8]:
original_image_paths = glob.glob('../input/siim-covid19-detection/train/*/*/*')

In [9]:
temp_df = pd.DataFrame()
temp_df['image_paths'] = original_image_paths
temp_df['image_id'] = temp_df['image_paths'].map(lambda x: x.split('/')[-1].split('.')[0])
temp_df['study_id'] = temp_df['image_paths'].map(lambda x: x.split('/')[-3])

meta_df = pd.read_csv('../input/siim-covid19-resized-to-512px-png/meta.csv')
meta_df = meta_df[meta_df['split'] == 'train']
meta_df.drop(columns='split', inplace=True)

meta_df = pd.merge(meta_df, temp_df, on='image_id')
meta_df.head(2)

,image_id,dim0,dim1,image_paths,study_id
0,d8ba599611e5,2336,2836,../input/siim-covid19-detection/train/cd5dd5e6...,cd5dd5e6f3f5
1,29b23a11d1e4,3488,4256,../input/siim-covid19-detection/train/49358afc...,49358afcfb80


In [10]:
study_df = pd.merge(study_df, meta_df, on='study_id')
study_df.drop(columns = ['dim0', 'dim1'], inplace=True)
study_df.columns = ['N', 'T', 'I', 'A', 'study_id', 'image_id', 'image_paths']
study_df.head(2)

,N,T,I,A,study_id,image_id,image_paths
0,0,1,0,0,00086460a852,65761e66de9f,../input/siim-covid19-detection/train/00086460...
1,0,0,0,1,000c9c05fd14,51759b5579bc,../input/siim-covid19-detection/train/000c9c05...


In [11]:
np.sum(study_df.isna())

N              0
T              0
I              0
A              0
study_id       0
image_id       0
image_paths    0
dtype: int64

In [12]:
def get_class(mat):
    final = []
    for row in mat:
        res = 0
        for i in row:
            if i==1:
                final.append(res)
            else:
                res += 1
    return final
            
study_df['label'] = get_class(study_df[['N', 'T', 'I', 'A']].values)
study_df.head(2)

,N,T,I,A,study_id,image_id,image_paths,label
0,0,1,0,0,00086460a852,65761e66de9f,../input/siim-covid19-detection/train/00086460...,1
1,0,0,0,1,000c9c05fd14,51759b5579bc,../input/siim-covid19-detection/train/000c9c05...,3


In [13]:
len(study_df)

6334

In [14]:
splitter = StratifiedKFold(random_state=1234)
study_df['kfold'] = -1

for fold, (trn_, val_) in enumerate(splitter.split(study_df.index, study_df['label'])):
    study_df.loc[val_, 'kfold'] = fold

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [15]:
study_df.kfold.value_counts()

0    1267
1    1267
2    1267
3    1267
4    1266
Name: kfold, dtype: int64

In [16]:
class_weights = compute_class_weight('balanced', study_df['label'].unique(), study_df['label'].values)
class_weights = {k:v for k,v in zip(study_df['label'].unique(), class_weights)}
class_weights

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[1 3 0 2], y=[1 3 0 ... 1 1 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


{1: 0.5266045892916528,
 3: 3.2784679089026914,
 0: 0.9121543778801844,
 2: 1.4291516245487366}

# Creating dataset

In [17]:
def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.with_options(ignore_order)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [18]:
GCS_DS_PATH

'gs://kds-058dc294a98b269b36cddd9dc62b407ecfff45d53408897614146080'

# Training Function

In [19]:
def run_fold(fold):
    
    df_train = study_df[study_df['kfold'] != fold]
    df_valid = study_df[study_df['kfold'] == fold]
    
    train_paths = GCS_DS_PATH + '/train/' + df_train['image_id'] + '.png'
    valid_paths = GCS_DS_PATH + '/train/' + df_valid['image_id'] + '.png'

    train_labels = df_train[['N', 'T', 'I', 'A']].values
    valid_labels = df_valid[['N', 'T', 'I', 'A']].values
    
    train_decoder = build_decoder(with_labels=True, target_size=(config.IMG_SIZE, config.IMG_SIZE), ext='png')
    valid_decoder = build_decoder(with_labels=False, target_size=(config.IMG_SIZE, config.IMG_SIZE), ext='.png')
    
    train_dataset = build_dataset(train_paths,
                                  train_labels,
                                  bsize = config.BATCH_SIZE,
                                  decode_fn = train_decoder)
    
    valid_dataset = build_dataset(valid_paths,
                                 valid_labels,
                                 bsize = config.BATCH_SIZE,
                                 decode_fn = train_decoder,
                                 repeat = False, 
                                 shuffle = False, 
                                 augment = False)
    
    tf.keras.backend.clear_session()
    
    with strategy.scope():
    
        model = tf.keras.Sequential([
            tf.keras.models.load_model('../input/pre-training-efficientnet/feature_extractor.h5'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(config.NUM_CLASSES, activation='sigmoid')
        ])
        
        # For starting iterations set base-model to non-trainable
        model.layers[0].trainable = False

        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                      loss = tf.keras.losses.BinaryCrossentropy(),
                      metrics = [tf.keras.metrics.AUC(multi_label=True)],
                      steps_per_execution = 32)
        
        
        
    lr_scheduler   = tf.keras.callbacks.ReduceLROnPlateau(patience=2, min_delta=0.005)
    early_stopping = tf.keras.callbacks.EarlyStopping(min_delta=0.005, patience=3)
    
    
    # train the last layer only
    history = model.fit(x = train_dataset,
                        validation_data = valid_dataset,
                        epochs = 3,
                        steps_per_epoch = len(df_train)//config.BATCH_SIZE,
                        callbacks = [lr_scheduler, early_stopping],
                        class_weight = class_weights)
    
    with strategy.scope():
        
        # Recompile the model with all layers trainable
        model.layers[0].trainable = True

        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005),
                      loss = tf.keras.losses.BinaryCrossentropy(),
                      metrics = [tf.keras.metrics.AUC(multi_label=True)],
                      steps_per_execution = 32)
        
    lr_scheduler   = tf.keras.callbacks.ReduceLROnPlateau(patience=1, min_delta=0.005)
    early_stopping = tf.keras.callbacks.EarlyStopping(min_delta=0.005, patience=2)
    
    
    # train the last layer only
    history = model.fit(x = train_dataset,
                        validation_data = valid_dataset,
                        epochs = config.EPOCHS,
                        steps_per_epoch = len(df_train)//config.BATCH_SIZE,
                        callbacks = [lr_scheduler, early_stopping],
                       class_weight = class_weights)
    
    model.save_weights(f'./EFNet_{fold}_weights.h5', overwrite=True)
    
    df = pd.DataFrame(history.history)
    df.to_csv(f'./history_{fold}.csv', index=False)
    
    return history

# Training the model
Cause of less Computational resources we will only run for 2 folds.

In [20]:
print('\n################################## Fold Started ##################################\n')
history = run_fold(4)
print('\n################################## Fold Ended ##################################\n')


################################## Fold Started ##################################

Epoch 1/3
39/39 [==============================] - 227s 6s/step - loss: 0.6099 - auc: 0.6437 - val_loss: 0.4982 - val_auc: 0.7175
Epoch 2/3
39/39 [==============================] - 9s 227ms/step - loss: 0.5315 - auc: 0.7091 - val_loss: 0.4608 - val_auc: 0.7296
Epoch 3/3
39/39 [==============================] - 9s 226ms/step - loss: 0.5134 - auc: 0.7067 - val_loss: 0.4587 - val_auc: 0.7297
Epoch 1/20
39/39 [==============================] - 370s 9s/step - loss: 0.4774 - auc_1: 0.7397 - val_loss: 0.4049 - val_auc_1: 0.7645
Epoch 2/20
39/39 [==============================] - 32s 817ms/step - loss: 0.4479 - auc_1: 0.7820 - val_loss: 0.4281 - val_auc_1: 0.7714
Epoch 3/20
39/39 [==============================] - 32s 817ms/step - loss: 0.4184 - auc_1: 0.8073 - val_loss: 0.3935 - val_auc_1: 0.7863
Epoch 4/20
39/39 [==============================] - 32s 814ms/step - loss: 0.4207 - auc_1: 0.8117 - val_loss: 0.38

In [29]:
# Different folds results
histories = []
for i in glob.glob('./*'):
    if 'hist' in i:
        histories.append(i)
        
avg_train_loss = 0
avg_train_auc = 0
avg_valid_loss = 0
avg_valid_auc = 0


histories.sort()
for i in histories:
    df = pd.read_csv(i)
    avg_train_loss += df['loss'].values[-1]
    avg_train_auc += df['auc_1'].values[-1]
    avg_valid_loss += df['val_loss'].values[-1]
    avg_valid_auc += df['val_auc_1'].values[-1]
    display(i, df)

'./history_0.csv'

,loss,auc_1,val_loss,val_auc_1,lr
0,0.482467,0.732068,0.420018,0.782751,0.000500
1,0.444282,0.782854,0.406793,0.789046,0.000500
2,0.427223,0.810353,0.395025,0.796630,0.000500
3,0.409399,0.821163,0.397320,0.792324,0.000500
4,0.386715,0.843230,0.379640,0.801579,0.000050
5,0.370825,0.854460,0.379577,0.802913,0.000050
6,0.370915,0.854891,0.385288,0.800456,0.000005


'./history_1.csv'

,loss,auc_1,val_loss,val_auc_1,lr
0,0.473880,0.745063,0.453873,0.767036,0.00050
1,0.446626,0.785836,0.398726,0.792399,0.00050
2,0.427190,0.805940,0.414682,0.783783,0.00050
3,0.400859,0.834836,0.403568,0.789905,0.00005


'./history_2.csv'

,loss,auc_1,val_loss,val_auc_1,lr
0,0.477014,0.745796,0.417344,0.769873,0.000500
1,0.447517,0.782795,0.397724,0.799069,0.000500
2,0.428383,0.805634,0.390871,0.783426,0.000500
3,0.407778,0.826917,0.391765,0.789119,0.000500
4,0.378843,0.849376,0.385293,0.793538,0.000050
5,0.377470,0.854564,0.385352,0.794274,0.000050
6,0.363486,0.861355,0.390578,0.794029,0.000005


'./history_3.csv'

,loss,auc_1,val_loss,val_auc_1,lr
0,0.482708,0.740220,0.431286,0.770902,0.00050
1,0.450024,0.780058,0.383109,0.802138,0.00050
2,0.430561,0.799063,0.383668,0.802617,0.00050
3,0.406851,0.826609,0.383822,0.802558,0.00005


'./history_4.csv'

,loss,auc_1,val_loss,val_auc_1,lr
0,0.475051,0.745809,0.404889,0.764501,0.000500
1,0.448860,0.782632,0.428110,0.771424,0.000500
2,0.419331,0.806847,0.393489,0.786250,0.000050
3,0.421435,0.813255,0.388793,0.787462,0.000050
4,0.413229,0.818441,0.391322,0.788827,0.000005


In [33]:
print('avg train loss: ', avg_train_loss/5)
print('avg train auc: ', avg_train_auc/5)
print('avg valid loss: ', avg_valid_loss/5)
print('avg valid auc: ', avg_valid_auc/5)

avg train loss:  0.39106777906417844
avg train auc:  0.8392262697219849
avg valid loss:  0.3909155786037445
avg valid auc:  0.7951549887657166
